In [1]:
# TODO: remove cell
import warnings
warnings.filterwarnings("ignore")
from articlecommon import *
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
lgbm = LGBMClassifier(random_state=42, verbose=-1)
psc_ohe = pd.get_dummies(X_le["Policy_Sales_Channel"], prefix="Policy_Sales_Channel")
X_le_psc = pd.concat([X_le, psc_ohe], axis=1)



psc_ohe = pd.get_dummies(X_le["Policy_Sales_Channel"], prefix="Policy_Sales_Channel")

counts = X_le["Policy_Sales_Channel"].value_counts()

counts = counts[counts == 1]
columns_to_remove = list(map(lambda x: "Policy_Sales_Channel_" + str(x), counts.index.tolist()))
psc_ohe = psc_ohe.drop(columns=columns_to_remove)

X_le_psc = pd.concat([X_le, psc_ohe], axis=1)

value_counts = X_le_psc["Policy_Sales_Channel"].value_counts().index
columns_to_remove = list(map(lambda x: "Policy_Sales_Channel_" + str(value_counts[x]), [*range(147, 86, -1)]))
columns_to_remove += ['Vintage']

from category_encoders import TargetEncoder
X_le_psc_te = X_le_psc.drop(columns=columns_to_remove).copy()
X_le_psc_te['Policy_Sales_Channel_te'] = X_le_psc_te['Policy_Sales_Channel']

X_le_psc_te_rc = X_le_psc_te.copy()
X_le_psc_te_rc['Region_Code_te'] = X_le_psc_te_rc['Region_Code']
selected_to_remove = pd.read_csv('results/columns_to_remov4.csv', index_col=0)["0"]

X_final = X_le_psc_te_rc
X_rc_ohe = X_final.drop(columns=selected_to_remove)
rc_ohe = pd.get_dummies(X_rc_ohe["Region_Code"], prefix="Region_Code")
X_rc_ohe = pd.concat([X_rc_ohe, rc_ohe], axis=1)

fixed_columns = [x for x in X_rc_ohe if x not in rc_ohe.columns]
columns_to_add = ["Region_Code_0.0", "Region_Code_1.0", "Region_Code_38.0", "Region_Code_52.0"]

encoder = TargetEncoder(cols=['Region_Code', 'Policy_Sales_Channel'])
# X_reduce_te = X_rc_ohe[fixed_columns + columns_to_add]
X_reduce_te = X_rc_ohe.loc[:, fixed_columns + columns_to_add]

all_columns = [x for x in X_reduce_te.columns.values if x not in ["Region_Code_0.0", "Region_Code_38.0", "Region_Code_52.0"]]



In [2]:
import optuna
np.seterr(all="ignore")

all_columns = [x for x in X_reduce_te.columns.values if x not in ["Region_Code_0.0", "Region_Code_38.0", "Region_Code_52.0"]]
transformer = ColumnTransformer(transformers=[('pwrtrans', PowerTransformer(), all_columns)])


def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1.0)
    max_depth = trial.suggest_int("max_depth", 2, 10)
    n_estimators = trial.suggest_int('n_estimators', 500, 1000)
    subsample = trial.suggest_uniform('subsample', 0.4, 1.0)
    te_smoothing = trial.suggest_loguniform("te_smoothing", 0.05, 10.0)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 7)
    gamma = trial.suggest_uniform("gamma", 0.0, 1.0)
    colsample_bytree = trial.suggest_uniform("colsample_bytree", 0.3, 1.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-3, 1e2)

    
    xgb = XGBClassifier(random_state=42, eval_metric="logloss", learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators, subsample=subsample, min_child_weight=min_child_weight, gamma=gamma, colsample_bytree=colsample_bytree, reg_alpha=reg_alpha)
    encoder = TargetEncoder(cols=['Policy_Sales_Channel', 'Region_Code'], smoothing=te_smoothing)
    model = make_pipeline(encoder, transformer, xgb)

    score = cross_val_score(model, X_reduce_te, y, cv=10, scoring='roc_auc', n_jobs=-1).mean() # TODO: set cv=10 and the whole dataset
    print("  ROC AUC: {}".format(score))

    print("  Params for trial " + str(trial.number) + ": ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    return score

In [3]:
study_name = 'xgb-study-higher-estimators'
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(study_name=study_name, direction="maximize", storage=storage_name)

study.enqueue_trial({
        "learning_rate": 0.021291366221284337,
        "max_depth": 8,
        "min_child_weight": 5,
        "n_estimators": 600,
        "subsample": 0.8706268352745364,
        "te_smoothing": 0.35648520716557985,
        "gamma": 0.20241066830358567,
        "colsample_bytree": 0.759012193204171,
        "reg_alpha": 0.010982331080557824
    })

optuna.logging.set_verbosity(optuna.logging.WARNING)
study.optimize(objective, n_trials=1)

print("\nBest trial:")
trial = study.best_trial

print("  ROC AUC: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-12-11 21:59:23,177] A new study created in RDB with name: xgb-study-higher-estimators


  ROC AUC: 0.8590063875903118
  Params for trial 0: 
    learning_rate: 0.021291366221284337
    max_depth: 8
    n_estimators: 600
    subsample: 0.8706268352745364
    te_smoothing: 0.35648520716557985
    min_child_weight: 5
    gamma: 0.20241066830358567
    colsample_bytree: 0.759012193204171
    reg_alpha: 0.010982331080557824

Best trial:
  ROC AUC: 0.8590063875903118
  Params: 
    colsample_bytree: 0.759012193204171
    gamma: 0.20241066830358567
    learning_rate: 0.021291366221284337
    max_depth: 8
    min_child_weight: 5
    n_estimators: 600
    reg_alpha: 0.010982331080557824
    subsample: 0.8706268352745364
    te_smoothing: 0.35648520716557985


In [ ]:
import optuna
study_name = 'xgb-study-higher-estimators'
storage_name = "sqlite:///{}.db".format(study_name)
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize",)
study.optimize(objective, n_trials=100)

  ROC AUC: 0.8586609478777143
  Params for trial 1: 
    learning_rate: 0.007501727456163487
    max_depth: 9
    n_estimators: 625
    subsample: 0.5476389740561816
    te_smoothing: 2.5406984480286567
    min_child_weight: 6
    gamma: 0.17893788092835738
    colsample_bytree: 0.9545336829344575
    reg_alpha: 0.027887635012464964
  ROC AUC: 0.8526636957272832
  Params for trial 2: 
    learning_rate: 5.24248511140162e-05
    max_depth: 9
    n_estimators: 935
    subsample: 0.7640872333247621
    te_smoothing: 0.06563040617595776
    min_child_weight: 7
    gamma: 0.09781756592320756
    colsample_bytree: 0.8548888519993834
    reg_alpha: 28.37888086895888
  ROC AUC: 0.8586926419357441
  Params for trial 3: 
    learning_rate: 0.0447497611481825
    max_depth: 5
    n_estimators: 849
    subsample: 0.9175834421279034
    te_smoothing: 1.078173694278031
    min_child_weight: 3
    gamma: 0.505760739727631
    colsample_bytree: 0.7535556905515772
    reg_alpha: 0.025450616913443688
  

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

  ROC AUC: 0.8536612014280038
  Params for trial 7: 
    learning_rate: 2.356607506018654e-06
    max_depth: 6
    n_estimators: 698
    subsample: 0.519880878617853
    te_smoothing: 0.3634009685164133
    min_child_weight: 7
    gamma: 0.5693014255834046
    colsample_bytree: 0.8146446677262096
    reg_alpha: 0.012130409439705345


/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sli

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sli

...

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

  ROC AUC: 0.8562589982864479
  Params for trial 24: 
    learning_rate: 0.029586513083842404
    max_depth: 10
    n_estimators: 734
    subsample: 0.47173753663534396
    te_smoothing: 0.7613757351789348
    min_child_weight: 6
    gamma: 0.341758331175832
    colsample_bytree: 0.7345927273717877
    reg_alpha: 0.2859692726075759


/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

  ROC AUC: 0.8584883823325148
  Params for trial 25: 
    learning_rate: 0.023659762947692196
    max_depth: 8
    n_estimators: 789
    subsample: 0.5898767060749524
    te_smoothing: 1.3858016287824326
    min_child_weight: 2
    gamma: 0.1980465010949707
    colsample_bytree: 0.6776670501723692
    reg_alpha: 0.05891564146677962
  ROC AUC: 0.8560917882996446
  Params for trial 26: 
    learning_rate: 0.0014126791483704583
    max_depth: 7
    n_estimators: 672
    subsample: 0.6649917174546917
    te_smoothing: 9.828453519080046
    min_child_weight: 5
    gamma: 0.43943941739494985
    colsample_bytree: 0.8160205405141328
    reg_alpha: 0.0023394973959754736
  ROC AUC: 0.8589888377324734
  Params for trial 27: 
    learning_rate: 0.010307959481883094
    max_depth: 9
    n_estimators: 549
    subsample: 0.7024040977573527
    te_smoothing: 2.784400549975954
    min_child_weight: 4
    gamma: 0.11421328456009706
    colsample_bytree: 0.5926772375621774
    reg_alpha: 0.1158490960644